<a href="https://colab.research.google.com/github/llNeeleshll/bert_sentence_clustring/blob/main/Bert_Sentence_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers
!pip install kmeans-pytorch

     |████████████████████████████████| 1.3MB 2.7MB/s 
     |████████████████████████████████| 1.1MB 18.2MB/s 
     |████████████████████████████████| 890kB 18.6MB/s 
     |████████████████████████████████| 2.9MB 22.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=950f16b4fa744dccecc22b34f28399cc01590f620b29a9ad7ca05a0f39579467
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
import torch
from transformers import BertTokenizer, BertModel
from scipy.spatial.distance import cosine
import numpy  as np
import pandas as pd

In [22]:
class BertEmbeddings:

  def __init__(self):
    self.model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )
    self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    
  def embed_text(self, text_to_embed):
    
    marked_text = "[CLS] " + text_to_embed + " [SEP]"
    tokenized_text = self.tokenizer.tokenize(marked_text)
    indexed_tokens = self.tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    with torch.no_grad():
      
      outputs = self.model(tokens_tensor, segments_tensors)
      # Evaluating the model will return a different number of objects based on 
      # how it's  configured in the `from_pretrained` call earlier. In this case, 
      # becase we set `output_hidden_states = True`, the third item will be the 
      # hidden states from all layers. See the documentation for more details:
      # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
      hidden_states = outputs[2]
      
    # Concatenate the tensors for all layers. We use `stack` here to
    # create a new dimension in the tensor.
    token_embeddings = torch.stack(hidden_states, dim=0)
    
    # Remove dimension 1, the "batches".
    token_embeddings = torch.squeeze(token_embeddings, dim=1)

    # Swap dimensions 0 and 1.
    token_embeddings = token_embeddings.permute(1,0,2)

    # Stores the token vectors, with shape [31 x 768]
    token_vecs_sum = []

    # For whole sentence
    token_vecs = hidden_states[-2][0]

    # Calculate the average of all 22 token vectors.
    sentence_embedding = torch.mean(token_vecs, dim=0)

    return sentence_embedding

    # For each token in the sentence...
    for token in token_embeddings:

        # `token` is a [12 x 768] tensor

        # Sum the vectors from the last four layers.
        sum_vec = torch.sum(token[-4:], dim=0)
        
        # Use `sum_vec` to represent `token`.
        token_vecs_sum.append(sum_vec)

    total_vector_sum = torch.zeros([1,768])

    for i in range(len(token_vecs_sum)):
        total_vector_sum += token_vecs_sum[i]

    return total_vector_sum


In [23]:
be = BertEmbeddings()

x = ["I hate it",
     "I really love to play cricket",
     "This is very bad. You should kill him", 
     "This is awesome, I am looking to work"]

#encoded_x = be.embed_text(x[1])
l = [be.embed_text(x[i]) for i in range(0, len(x))]

#print(encoded_x.shape)

l = tuple(l)
encoded_x = torch.stack((l), dim=0)
print(encoded_x.data)
print(encoded_x.shape)

#et  = be.embed_text("deposite money in a bank")
#et2 = be.embed_text("This is the best")

tensor([[ 0.0439,  0.6635, -0.1679,  ...,  0.2582,  0.1644,  0.3177],
        [ 0.6989,  0.0863,  0.0043,  ..., -0.1464,  0.1625, -0.4249],
        [ 0.0784,  0.1402, -0.0037,  ..., -0.0830,  0.1059,  0.0242],
        [ 0.1594,  0.1144,  0.3340,  ..., -0.3674,  0.0539,  0.1564]])
torch.Size([4, 768])


In [24]:
from kmeans_pytorch import kmeans

cluster_ids_x, cluster_centers = kmeans(
    X=encoded_x, num_clusters=2, distance='euclidean')

print(cluster_ids_x)

[running kmeans]: 2it [00:00, 426.16it/s, center_shift=0.000000, iteration=2, tol=0.000100]

running k-means on cpu..
tensor([0, 1, 0, 1])


In [15]:
# read the email file
df = pd.read_excel("sentiment_review.xlsx")
df.head()
x = df.iloc[:,0].tolist()
print(x[5])

This isn't the comedic Robin Williams, nor is it the quirky/insane Robin Williams of recent thriller fame. This is a hybrid of the classic drama without over-dramatization, mixed with Robin's new love of the thriller. But this isn't a thriller, per se. This is more a mystery/suspense vehicle through which Williams attempts to locate a sick boy and his keeper.<br /><br />Also starring Sandra Oh and Rory Culkin, this Suspense Drama plays pretty much like a news report, until William's character gets close to achieving his goal.<br /><br />I must say that I was highly entertained, though this movie fails to teach, guide, inspect, or amuse. It felt more like I was watching a guy (Williams), as he was actually performing the actions, from a third person perspective. In other words, it felt real, and I was able to subscribe to the premise of the story.<br /><br />All in all, it's worth a watch, though it's definitely not Friday/Saturday night fare.<br /><br />It rates a 7.7/10 from...<br /><

In [25]:
# read the email file
df = pd.read_excel("sentiment_review.xlsx")
x = df.iloc[:3,0].tolist()

embeding_bert = BertEmbeddings()

# r = embeding_bert.embed_text("Subject: here ' s a hot play in motion  homeland security investments  the terror attacks on the united states on september 11 , 20 ol have  changed  the security landscape for the foreseeable future . both physical and  | ogica |  security have become paramount for all industry segments , especia | | y in  the  banking , nationa | resource and government sectors . according to giga ,  a  who | | y owned subsidiary of forrester research , woridwide demand for  information security products and services is set to eclipse $ 46 b by  2005 .  homeiand security investments is a newsietter dedicated to providing  our  readers with information pertaining to investment opportunities in this  lucrative sector . as we know , events related to homeland security  happen  with lightning speed . what we as investors can do is position  ourselves in  such a way as to take advantage of the current trends and be ready to  capitalize on events which have yet to happen . homeland security  investments is here to heip our readers do just that .  with this in mind , it is with great excitement that we present vinoble ,  inc .  this stock is expected to do big things in both the near and | ong  terms .  symbol : vnbl . ob  current price : o . 08  short term target price : o . 35  12 month target price : 1 . 20  * * * why we believe vnbl . ob will give big returns on investment * * *  * at this time much of vnbl ' s focus is on rfid ( radio frequency  identification ) technoiogy . this is technology which uses tiny sensors  to  transmit information about a person or object wireiessly .  * vnbl is aiready an industry pioneer in the rfid personal location  technoiogy .  * vnbl is developing a form of rfid technology which allows companies  and  governments to wirelessly track their assets and resources . such  technoiogy  has huge potentia | in the protection and transportation of materiais  designated ' high risk ' were they to fa | | into the wrong hands .  * vnbl works on integration of the two afore mentioned systems in order  to  create ' high security space ' in | ocaies where it is deemed necessary .  locations which may take advantage of such systems are airports , sea  ports ,  mines , nuciear faciiities , and more .  * as with a | | stocks , news drives the short term price . fresh news has  made vnbl a hot buy .  news on vnbl  malibu , calif . - - ( business wire ) - - june 16 , 2 oo 5 - - vinoble , inc .  ( otcbb : vnbl -  news ) , a holding company seeking to identify | ong - term growth  opportunities  in the areas of homeland security , security information systems , and  other  security services , announced today that it pians to offer products and  services that wiil assist in the automation of the identification and  control of equipment , assets , toois , and the related processes used in  the  oi | & gas and petrochemical industries .  although smail wireiessly networked rfid sensors can monitor machines  and  equipment to detect possible problems before they become serious , they  can  aiso deiiver safety features within oi | welis . oi | maybe trapped in  different | ayers of rock , aiong with gas and water . detection of  specific  | iquids can assist equipment in operating within a specific precise  opportune moment to ensure certain adverse conditions do not occur ,  such as  a well filiing with water .  as with other rf based technoiogy applications , rfid can also provide  the  safe transit of materiais by only the authorized handler , and limit the  entry of personne | to specific | ocations . ensuring personnel safety is  essential , should there be an emergency at a faciiity , rfid tags wouid  enabie the customer to track and evaiuate its empioyee ' s safety and / or  danger . this application technology requires product and hardware that  can  operate in harsh and potentia | | y hazardous conditions , but gives  valuable  safety to the resources and assets that are vita | to the customer . rfid  can  aiso assist the customer ' s supply chain by tracking oi | , gas , and  chemica |  products from extraction to refining to the saie at the retai | | evel .  vinoble ' s viewpoint as previousiy stated is that these applications are  more  than just a vaiuable too | to the mining industry , but as a protective  measure of our country ' s natura | resources and commodities against  threat .  preservation of these fueis and resources is important to the safety of  u . s .  industry and economy .  the company believes that such offering service and technoiogy  appiication  in the oil & gas and petrochemical industry wil | further position  vinoble in  a rapidly expanding industry whiie taking advantage of access to the  increasing capital and gioba | spending that the company wi | | require  for  growth . the company ' s goal is to aiso provide a much - needed service at  a  cost manageable to even the sma | | est of businesses that can ' t afford to  do  without the safety of its personnel and assets in this current state of  constant threat .  this is outstanding news . the growth potential for this company is  exceptional . in an already hot industry , vnbl . ob stands out as a truiy  innovative pioneer . we see big things happening to this stock .  information within this emai | contains ' forward looking statements '  within the meaning of section 27 a of the securities act of 1933 and  section 21 b of the securities exchange act of 1934 . any statements that  express or involve discussions with respect to predictions ,  expectations , beliefs , pians , projections , objectives , goals ,  assumptions or  future  events or performance are not statements of historica | fact and may be  ' forward | ooking statements . ' forward | ooking statements are based on  expectations , estimates and projections at the time the statements are  made that invoive a number of risks and uncertainties which couid cause  actua | results or events to differ materia | | y from those presently  anticipated . forward looking statements in this action may be  identified  through the use of words such as ' projects ' , ' foresee ' , ' expects ' ,  ' wi | | , ' ' anticipates , ' ' estimates , ' ' beiieves , ' ' understands ' or  that by  statements indicating certain actions ' may , ' ' couid , ' or ' might ' occur .  as with many micro - cap stocks , today ' s company has additional risk  factors worth noting . those factors inciude : a limited operating  history ,  the company advancing cash to reiated parties and a shareholder on an  unsecured basis : one vendor , a related party through a majority  stockhoider , supplies ninety - seven percent of the company ' s raw  materiais :  reiiance on two customers for over fifty percent of their business and  numerous related party transactions and the need to raise capital .  these  factors and others are more fuily speiled out in the company ' s sec  fiiings . we urge you to read the filings before you invest . the rocket  stock  report does not represent that the information contained in this  message states ail materia | facts or does not omit a material fact  necessary  to make the statements therein not misleading . ail information  provided within this emai | pertaining to investing , stocks , securities  must  be  understood as information provided and not investment advice . the  rocket stock report advises all readers and subscribers to seek advice  from  a registered professiona | securities representative before deciding to  trade in stocks featured within this email . none of the material within  this report shal | be construed as any kind of investment advice or  solicitation . many of these companies are on the verge of bankruptcy .  you  can lose ail your money by investing in this stock . the publisher of  the rocket stock report is not a registered investment advisor .  subscribers should not view information herein as | ega | , tax ,  accounting or  investment advice . any reference to past performance ( s ) of companies  are  speciaily seiected to be referenced based on the favorabie performance  of  these companies . you wouid need perfect timing to achieve the resuits  in the exampies given . there can be no assurance of that happening .  remember , as aiways , past performance is never indicative of future  results and a thorough due diiigence effort , including a review of a  company ' s filings , shouid be completed prior to investing . in  compiiance  with the securities act of 1933 , section 17 ( b ) , the rocket stock report  discioses the receipt of tweive thousand doilars from a third party  ( gem , inc . ) , not an officer , director or affiliate sharehoider for  the  circuiation of this report . gem , inc . has a position in the stock  they  wil | se | | at any time without notice . be aware of an inherent confiict  of interest resuiting from such compensation due to the fact that this  is a paid advertisement and we are conflicted . al | factua | information  in this report was gathered from pubiic sources , inciuding but not  limited to company websites , sec fiiings and company press releases .  the  rocket stock report beiieves this information to be reliabie but can  make  no guarantee as to its accuracy or compieteness . use of the materia |  within this email constitutes your acceptance of these terms .")

# print(r)

# flag = 0
# for k in range(len(x)):
#   try:
#     embeding_bert.embed_text(x[k])
#   except Exception as e:
#     flag += 1
    
#     print("Broke " + str(flag))

# embed the list
l = [embeding_bert.embed_text(x[i]) for i in range(0, len(x))]

# stack the embeding vector to pass to k means
l = tuple(l)
encoded_x = torch.stack((l), dim=0)

print(encoded_x.data)
print(encoded_x.shape)

RuntimeError: ignored

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from scipy.spatial.distance import cosine

# Calculate the cosine similarity between the sentences 

sim = (1 - cosine(et, et2)) * 100
from kmeans_pytorch import kmeans
print("The similarity is : %.2f" % sim, "%")

In [ ]:
from kmeans_pytorch import kmeans_predict

ex = "i dont like food"
ex = be.embed_text(ex)
print(ex.shape)
cluster_ids_y = kmeans_predict(
    ex , cluster_centers, 'euclidean'
)

torch.Size([1, 768])
predicting on cpu..


IndexError: ignored

In [ ]:
plt.figure(figsize=(4, 3), dpi=160)
plt.scatter(x[:, 0], x[:, 1], c=cluster_ids_x, cmap='cool')
plt.scatter(y[:, 0], y[:, 1], c=cluster_ids_y, cmap='cool', marker='X')
plt.scatter(
    cluster_centers[:, 0], cluster_centers[:, 1],
    c='white',
    alpha=0.6,
    edgecolors='black',
    linewidths=2
)
plt.axis([-1, 1, -1, 1])
plt.tight_layout()
plt.show()


In [ ]:
s1 = torch.empty(0,3)
s2 = torch.tensor([4,5,6])
s2 = torch.stack((s1,s2), dim=0)

print(s2)

RuntimeError: ignored

In [ ]:
be = BertEmbeddings()
import numpy  as np
x1 = ["I hate it",
     "I really love to play cricket",
     "This is very bad. You should kill him", 
     "This is awesome, I am looking to work"]

#encoded_x = be.embed_text(x[1])
l1 = [be.embed_text(x1[i]) for i in range(0, len(x1))]
print(len(l1))
numy = np.array(l1)
#print(encoded_x.shape)

l1 = tuple(l1)
encoded_x = torch.stack((l1), dim=0)
print(encoded_x.data)
print(encoded_x.shape)

#et = be.embed_text("deposite money in a bank")
#et2 = be.embed_text("This is the best")

4


ValueError: ignored

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=0).fit(numy)


NameError: ignored